In [1]:
import glob
import cv2
import os
import numpy as np

In [2]:
#images = [cv2.imread(file) for file in glob.glob("Photos/*.jpg")]

In [3]:
subjects = ["","Nishant"]

In [4]:
def detect_face(img):
     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
     face_cascade = cv2.CascadeClassifier(r'C:\Users\nischugh\Anaconda3\pkgs\libopencv-3.4.1-h875b8b8_3\Library\etc\lbpcascades\lbpcascade_frontalface.xml')
     faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
     #print(faces)
     grays = []
     if (len(faces) == 0):
        return None, None
     if (len(faces) == 1):
            (x, y, w, h) = faces[0]
            return gray[y:y+w, x:x+h], faces[0]
     else:
        for i in range(len(faces)):
            (x, y, w, h) = faces[i]
            grays.append([gray[y:y+w, x:x+h]])
        return gray, faces

In [5]:
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
#    print(dirs)
    for dir_name in dirs:
        if not dir_name.startswith("S"):
            continue;
        label = int(dir_name.replace("S",""))
        subject_dir_path = data_folder_path + "/" + dir_name
        #print(subject_dir_path)
        subject_images_names = os.listdir(subject_dir_path)
        #print(subject_images_names)
        for image_name in subject_images_names:
            #print(image_name)
            if image_name.startswith("."):
                continue;
            image_path = subject_dir_path + "/" + image_name
            #print(image_path)
            image = cv2.imread(image_path)
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            face, rect = detect_face(image)
            #print(face)
            if face is not None:
                faces.append(face)
                labels.append(label)
            cv2.destroyAllWindows()
            cv2.waitKey(10)
            cv2.destroyAllWindows()
    return faces, labels

In [6]:
print("Preparing data...")
faces, labels = prepare_training_data("Pics/")
print("Data prepared")
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))



Preparing data...
Data prepared
Total faces:  31
Total labels:  31


In [7]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))

In [17]:
def draw_rectangle(img, face):
    #print(img)
    #print(len(face))
    for (x,y,w,h) in face:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [22]:
def predict(test_img):
    img = test_img.copy()
    rect, face = detect_face(img)
    print(face)
    #print(len(rect))
    label = face_recognizer.predict(face)
    print(label)
    label_text = subjects[label[0]]
    draw_rectangle(img, face)
    print(rect[0][0])
    print(rect[1][1]-5)
    draw_text(img, label_text, rect[0][0], rect[1][1]-5)
    return img

In [29]:
print("Predicting images...")
test_img1 = cv2.imread("test5.jpg")
#test_img2 = cv2.imread("test.jpg")
predicted_img1 = predict(test_img1)
#predicted_img2 = predict(test_img2)
print("Prediction complete")
cv2.imshow(subjects[1], predicted_img1)
#cv2.imshow(subjects[1], predicted_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

Predicting images...
[[ 968 2142  508  508]
 [ 288 1696  773  773]
 [2029 1133   53   53]]
(1, 0.0)
3
148
144
Prediction complete
